In [1]:
import claudio
import librosa
import numpy as np
import os
import pandas
import wcqtlib.config as C
import wcqtlib.common.utils as utils

%matplotlib inline
import seaborn
import mpld3

seaborn.set()
np.set_printoptions(precision=4, suppress=True)
mpld3.enable_notebook()

config = C.Config.from_yaml("./data/master_config.yaml")

/Users/cjacoby/miniconda3/envs/ismir2016/lib/python3.5/site-packages/librosa/core/audio.py:37: UserWarning: Could not import scikits.samplerate. Falling back to scipy.signal
  warnings.warn('Could not import scikits.samplerate. '


In [8]:
extract_path = os.path.expanduser(config["paths/extract_dir"])
datasets_file = config['dataframes/datasets']
datasets_path = os.path.join(extract_path, datasets_file)
datasets_df = pandas.read_json(datasets_path)
notes_file = config['dataframes/notes']
notes_path = os.path.join(extract_path, notes_file)
notes_df = pandas.read_pickle(notes_path)
notes_df = notes_df
features_df_fn = config['dataframes/features']
features_path = os.path.join(extract_path, features_df_fn)
features_df = pandas.read_pickle(features_path)

In [ ]:
import wcqtlib.data.parse
new_notes_df = wcqtlib.data.parse.normalize_instrument_names(notes_df)
new_notes_df.to_pickle(notes_path)
new_features_df = wcqtlib.data.parse.normalize_instrument_names(features_df)
new_features_df.to_pickle(features_path)

In [ ]:
print(notes_df["instrument"].unique())
print(features_df["instrument"].unique())

In [ ]:
import matplotlib.pyplot as plt
def viz_audio(record, downsample_rate=150):
    """Plot the audio from a record"""
    audio_path = record["audio_file"][0]
    audio, sr = claudio.read(audio_path, channels=1)
    print("N samples: {}\nDuration: {:.3f}s\nsr: {}".format(audio.shape[0], len(audio)/sr, sr))
    
    plt.figure(figsize=(6, 2))
    print(audio.shape)
    plt.plot(audio[::downsample_rate])
    

def play_audio(record):
    """Play the audio from a record"""
    import claudio.sound
    audio, sr = claudio.read(path, channels=1)
    claudio.sound.soundsc(audio, sr)

    
def viz_features(record):
    """Vizualize the features from a record."""
    import librosa.display
    cqt_path = record["cqt"][0]
    cqt_feats = np.load(cqt_path)['cqt']
    cqt = cqt_feats.reshape((cqt_feats.shape[1], cqt_feats.shape[2]))
    print("CQT Shape:", cqt.shape)
    
    plt.figure(figsize=(8, 4))
#     librosa.display.specshow(cqt, y_axis="cqt_hz")
    plt.imshow(np.log(cqt), aspect="auto", interpolation="none", cmap="OrRd")
    plt.colorbar(format='%+2.0f dB')
    plt.title('Constant-Q power spectrogram (Hz)')

In [ ]:
sample = features_df.sample()
path = sample['audio_file'][0]
dataset = sample['dataset'][0]
instrument = sample['instrument'][0]
print("Path:", os.path.basename(path), "from", dataset, "instrument:", instrument)

viz_audio(sample)
viz_features(sample)

play_audio(sample)

In [ ]:
import wcqtlib.data.cqt as cqt
#cqt.cqt_from_df(data_root, extract_path, notes_df_fn, features_df_fn)

In [10]:
def check_instruments(df):
    print(df["instrument"].unique())

In [13]:
print(notes_df)
print("All:", sorted(check_instruments(notes_df)))
# print("RWC:", sorted(check_instruments(notes_df[notes_df["dataset"] == "rwc"])))
# print("RWC:", sorted(check_instruments(notes_df[notes_df["dataset"] == "uiowa"])))
# print("RWC:", sorted(check_instruments(notes_df[notes_df["dataset"] == "philharmonia"])))

                                                            audio_file  \
p00066b84 p00066b84                                              False   
p0009a0d8 p0009a0d8                                              False   
p0009ba67 p0009ba67                                              False   
p000d4dfe p000d4dfe                                              False   
p003eeaaf p003eeaaf                                              False   
p0041e1c4 p0041e1c4                                              False   
p00574656 p00574656                                              False   
p0065d464 p0065d464                                              False   
p006655e9 p006655e9  /Users/cjacoby/data/ismir2016-wcqt-data/philha...   
p00674c0e p00674c0e                                              False   
p0072df1b p0072df1b                                              False   
p0074f368 p0074f368                                              False   
p0093e4b3 p0093e4b3                   

TypeError: 'NoneType' object is not iterable

In [ ]:
np.random.randint(2, size=12)